In [1]:
require(data.table)
require(magrittr)
require(sensitivity)

Loading required package: data.table
Loading required package: magrittr
Loading required package: sensitivity
Registered S3 method overwritten by 'sensitivity':
  method    from 
  print.src dplyr


In [2]:
z.ranges <- fread("input-ranges.tsv")
z.ranges %>% dim

[1] 58  6

In [3]:
z.ranges[, `:=`(Minimum = 0.6 * Default, Maximum = 1.6 * Default)]
z.ranges[Variable == "random stream", `:=`(Minimum = 0, Maximum = 10)]

In [4]:
z.design <- morris(
    NULL,
    factors = z.ranges$Variable,
    r = 100,
    design = list(
        type = "oat",
        levels = mapply(function(t, x0, x1) {
            if (t == "Integer")
                x1 - x0 + 1
            else if (t == "Boolean")
                2
            else
                5
        }, z.ranges$Type, z.ranges$Minimum, z.ranges$Maximum),
        grid.jump = 1
    )
)
z.design$X %>% dim

[1] 5900   58

In [5]:
write.table(z.design$X, file = "design.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

In [6]:
z.inputs <- cbind(
    Run = 1:(dim(z.design$X)[1]),
    data.table(
        sweep(
            sweep(z.design$X, MARGIN = 2, z.ranges$Maximum - z.ranges$Minimum, `*`),
            MARGIN = 2,
            z.ranges$Minimum,
            `+`
        )
    )
)
z.inputs %>% summary

      Run       advertising budget aversion to NPV deviation
 Min.   :   1   Min.   :300000     Min.   :0.120            
 1st Qu.:1476   1st Qu.:425000     1st Qu.:0.170            
 Median :2950   Median :550000     Median :0.220            
 Mean   :2950   Mean   :539407     Mean   :0.219            
 3rd Qu.:4425   3rd Qu.:675000     3rd Qu.:0.270            
 Max.   :5900   Max.   :800000     Max.   :0.320            
 base external investor ask rate bioproduct long term price
 Min.   : 4.800                  Min.   :3000              
 1st Qu.: 6.800                  1st Qu.:4250              
 Median : 8.800                  Median :5500              
 Mean   : 8.999                  Mean   :5287              
 3rd Qu.:10.800                  3rd Qu.:6750              
 Max.   :12.800                  Max.   :8000              
 bioproduct offtake agreement bioproduct performance advantage
 Min.   :0.3000               Min.   :0.600                   
 1st Qu.:0.4250            

In [7]:
write.table(z.inputs, file="inputs.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

In [6]:
z.outputs <- fread("outputs.tsv")
z.outputs[Time == 2050] %>% summary

      Run            Time      abandoning bioproduct actual net income   
 Min.   :   1   Min.   :2050   Min.   :0             Min.   :-1.054e+09  
 1st Qu.:1490   1st Qu.:2050   1st Qu.:0             1st Qu.: 0.000e+00  
 Median :3018   Median :2050   Median :0             Median : 0.000e+00  
 Mean   :2976   Mean   :2050   Mean   :0             Mean   : 8.467e+08  
 3rd Qu.:4462   3rd Qu.:2050   3rd Qu.:0             3rd Qu.: 0.000e+00  
 Max.   :5900   Max.   :2050   Max.   :0             Max.   : 9.175e+09  
 bioproduct favorability indicator bioproduct market share economic
 Min.   :0                         Min.   :-31.4743                
 1st Qu.:0                         1st Qu.:  0.3758                
 Median :0                         Median :  0.5077                
 Mean   :0                         Mean   :  0.8113                
 3rd Qu.:0                         3rd Qu.:  0.6740                
 Max.   :0                         Max.   : 30.0885                
 biopr

In [8]:
z.outputs[, .N, by=Run][N != 8]

Run,N
<int>,<int>
356,5
357,5
358,5
359,5
360,5
768,3
769,3
770,3
771,3


In [ ]:
?morris